# Model South Korea Time Series for COVID: Publish for app

This notebook use multistep time serie model  (predicting number of cases future next days).

It converts Tensorflow model into TensorFlow Lite to be able to use it in a Lambda fonction on AWS.

After that, the lite model is tested and publish on AWS

Finally, the lambda function is tested

## Convert model in TFlite

**IMPORTANT TO DO manually:**
- **Update TRAIN_SPLIT in my_helpers.model module**

### import

In [1]:
# data useful lib
import pandas as pd
import numpy as np

# helper lib
import shutil
import os, stat
import re
import datetime
import math

# read json from http
import json
import urllib.request

# read csv from http
import io
import requests

# model lib
import tensorflow as tf

# from project
from my_helpers.data_plots_kr import load_df_feat_kr
from my_helpers.model import prepare_to_lambda, retrieve_from_lambda
from my_helpers.model import prepare_to_lambda_future
from my_helpers.model import create_list_past_hist, predict_list

from my_helpers.model_kr import prepare_data_features_kr
from my_helpers.model_kr import prepare_dataset_kr

from my_helpers.utils import clean_file

import settings

In [2]:
tf.__version__

'2.5.0'

### Definitions

In [3]:
PATH_TO_SAVE_DATA = settings.PATH_TO_SAVE_DATA

from my_helpers.model_kr import PATH_MDL_MULTI_STEP_KR
from my_helpers.model_kr import PATH_MDL_MULTI_TFLITE_KR 
from my_helpers.model_kr import PATH_MDL_MULTI_TFLITE_FILE_KR
from my_helpers.model_kr import PATH_SERVERLESS_KR
from my_helpers.model_kr import URL_PREDICT_KR
from my_helpers.model_kr import PAST_HISTORY # days used to predict next values in future
from my_helpers.model_kr import FUTURE_TARGET  # predict 3 days later
from my_helpers.model_kr import STEP 

from my_helpers.data_plots_kr import PATH_DF_FEAT_KR

date_format = "%Y-%m-%d"

#NB_POS_DATE_MIN_DF_FEAT = 140227 # on 12/05/2020

# plot
NB_DAY_PLOT = FUTURE_TARGET*9

# train split
from my_helpers.model_kr import TRAIN_SPLIT
print(f"TRAIN_SPLIT = {TRAIN_SPLIT}")


TRAIN_SPLIT = 347


### helper functions

### load data

In [4]:
# load
df_feat_kr = load_df_feat_kr()
# clean
df_feat_kr = prepare_data_features_kr(df_feat_kr)
# split train/test
df_feat_kr["train"] = [True if I <= TRAIN_SPLIT else False \
                       for I in range(df_feat_kr.shape[0])]
df_feat_kr

,nb_cases,nb_tests,nb_deaths,date,day_num,Jeju,Gyeongnam,Gyeongbuk,Jeonnam,Jeonbuk,...,Rt_Jeonbuk,sum_Jeonnam,Rt_Jeonnam,sum_Gyeongbuk,Rt_Gyeongbuk,sum_Gyeongnam,Rt_Gyeongnam,sum_Jeju,Rt_Jeju,train
date,,,,,,,,,,,,,,,,,,,,,
2020-04-03,10062.0,424365.0,174.0,2020-04-03,5.0,0.0,1.0,5.0,1.0,1.0,...,5.000000,10.0,inf,106.0,4.240000,20.0,20.000000,5.0,inf,True
2020-04-04,10156.0,434888.0,177.0,2020-04-04,6.0,0.0,1.0,1.0,0.0,0.0,...,5.000000,10.0,inf,67.0,1.030769,21.0,21.000000,5.0,inf,True
2020-04-05,10237.0,441662.0,183.0,2020-04-05,0.0,3.0,1.0,4.0,0.0,1.0,...,6.000000,9.0,9.000000,60.0,0.789474,21.0,10.500000,8.0,inf,True
2020-04-06,10284.0,447509.0,186.0,2020-04-06,1.0,0.0,2.0,2.0,0.0,0.0,...,6.000000,9.0,9.000000,60.0,0.769231,22.0,7.333333,8.0,inf,True
2020-04-07,10331.0,456654.0,192.0,2020-04-07,2.0,0.0,1.0,1.0,0.0,0.0,...,6.000000,9.0,9.000000,60.0,0.759494,22.0,5.500000,8.0,inf,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-06,144152.0,9823232.0,1973.0,2021-06-06,0.0,19.0,17.0,10.0,9.0,0.0,...,0.554217,128.0,0.558952,162.0,0.589091,269.0,0.838006,203.0,1.362416,False
2021-06-07,144637.0,9835771.0,1974.0,2021-06-07,1.0,13.0,28.0,9.0,3.0,3.0,...,0.565217,117.0,0.500000,159.0,0.593284,280.0,0.869565,197.0,1.279221,False
2021-06-08,145090.0,9869323.0,1975.0,2021-06-08,2.0,8.0,48.0,7.0,8.0,4.0,...,0.584416,118.0,0.510823,154.0,0.581132,314.0,0.963190,188.0,1.278912,False


In [5]:
df_feat_kr[df_feat_kr["train"] == False].head(5)

,nb_cases,nb_tests,nb_deaths,date,day_num,Jeju,Gyeongnam,Gyeongbuk,Jeonnam,Jeonbuk,...,Rt_Jeonbuk,sum_Jeonnam,Rt_Jeonnam,sum_Gyeongbuk,Rt_Gyeongbuk,sum_Gyeongnam,Rt_Gyeongnam,sum_Jeju,Rt_Jeju,train
date,,,,,,,,,,,,,,,,,,,,,
2021-03-17,96839.0,7088038.0,1686.0,2021-03-17,3.0,1.0,45.0,4.0,0.0,21.0,...,0.767241,34.0,0.453333,102.0,0.573034,350.0,4.605263,33.0,1.064516,False
2021-03-18,97284.0,7128355.0,1688.0,2021-03-18,4.0,2.0,35.0,5.0,0.0,15.0,...,0.844828,33.0,0.458333,100.0,0.617284,375.0,4.746835,32.0,1.103448,False
2021-03-19,97747.0,7176590.0,1690.0,2021-03-19,5.0,0.0,42.0,4.0,0.0,15.0,...,0.948276,33.0,0.532258,98.0,0.671233,414.0,5.671233,30.0,1.071429,False
2021-03-20,98194.0,7215917.0,1693.0,2021-03-20,6.0,1.0,31.0,11.0,0.0,16.0,...,1.089286,30.0,0.545455,96.0,0.685714,440.0,6.285714,26.0,0.838710,False
2021-03-21,98650.0,7234391.0,1696.0,2021-03-21,0.0,1.0,54.0,5.0,0.0,11.0,...,1.174312,24.0,0.452830,88.0,0.602740,491.0,7.439394,23.0,0.718750,False


### Prepare dataset

In [6]:
dataset, data_std, data_mean = prepare_dataset_kr(df_feat_kr)
print(dataset)
print("data_mean : ", data_mean)
print("data_std : ", data_std)

[[-0.43842911 -0.03335956 -1.13417801 ... -0.54278014  0.99641704
  -0.36171153]
 [-0.43518827 -0.12215801 -1.30377381 ... -0.54175984  1.49462556
  -0.36171153]
 [-0.69121488 -0.47024795 -1.81069496 ... -0.78082929 -1.49462556
  -0.36171153]
 ...
 [ 0.93244756  1.18850717  0.0758542  ...  0.92677324 -0.49820852
  -0.64043942]
 [ 0.92920672  1.36610408 -0.46955843 ...  1.59532597  0.
  -0.53435665]
 [ 0.96161515  1.04998158  0.24964907 ...  0.90279615  0.49820852
  -0.5718529 ]]
data_mean :  [9.74274736e+00 1.91463977e+01 6.28461287e+01 1.83575408e+01
 2.47927954e+02 1.90186801e+04 3.00000000e+00 4.41442174e+01]
data_std :  [1.02853891e+01 9.38454775e+00 1.42888758e+01 6.26324047e+00
 3.18936079e+02 1.56816352e+04 2.00719168e+00 6.20951246e+00]


In [7]:
dataset.shape[1]

8

In [8]:
PAST_HISTORY

14

In [9]:
dataset.shape[1]

8

### Load model

In [10]:
%%time
# reload best model
multi_step_model = tf.keras.models.load_model(PATH_MDL_MULTI_STEP_KR)

CPU times: user 1.61 s, sys: 93.3 ms, total: 1.7 s
Wall time: 1.85 s


In [11]:
multi_step_model.inputs[0].dtype

tf.float32

In [12]:
run_model = tf.function(lambda x: multi_step_model(x))
# This is important, let's fix the input size.
INPUT_SIZE = dataset.shape[1]
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([1, PAST_HISTORY, INPUT_SIZE],
                  multi_step_model.inputs[0].dtype))

# model directory.
MODEL_DIR = PATH_TO_SAVE_DATA + "/" + "keras_lstm"
multi_step_model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)

'''converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
converter.allow_custom_ops=True'''

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/keras_lstm/assets


### Save model TFlite

In [13]:
clean_file(PATH_MDL_MULTI_TFLITE_FILE_KR)

File /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/converted_model_20210618_10_09_55.tflite moved!


In [14]:
open(PATH_MDL_MULTI_TFLITE_FILE_KR, "wb").write(tflite_model)

16684

## Test converted model

### Predict with TF model (not-converted one)

#### Past days

In [15]:
# TENSORFLOW MODEL :
# prepare list of past histories
list_x = create_list_past_hist(dataset)
# predict
y_multi_pred = predict_list(list_x, multi_step_model)
# convert in positive cases
y_pos_pred = (y_multi_pred * data_std[4]) + data_mean[4] 
y_pos_pred

[357 - 371]
[364 - 378]
[371 - 385]
[378 - 392]
[385 - 399]
[392 - 406]
[399 - 413]
[406 - 420]
[413 - 427]
9


array([[613.6644 , 648.78076, 630.8229 , 640.5824 , 560.076  , 589.45905,
        589.4885 , 599.4492 , 622.3203 , 612.393  , 620.29285, 561.33026,
        582.824  , 582.336  , 700.3687 , 762.6068 , 734.11414, 737.3379 ,
        617.61523, 671.828  , 679.07764, 663.67224, 719.06104, 692.5442 ,
        695.2577 , 587.1393 , 633.1209 , 638.65125, 616.40076, 664.9818 ,
        638.49927, 644.74646, 545.38464, 581.7986 , 582.6671 , 651.83386,
        704.9096 , 680.4376 , 685.52637, 574.2683 , 624.26697, 629.7843 ,
        595.51013, 612.8972 , 610.70374, 616.7048 , 554.61865, 588.5552 ,
        591.2001 , 594.70044, 629.8554 , 613.10626, 618.7514 , 535.37354,
        570.31244, 570.8958 , 569.1732 , 584.3329 , 581.26025, 588.7269 ,
        530.13   , 559.1876 , 559.6262 ]], dtype=float32)

#### Future days

In [16]:
# prepare data : very last days
x_for_future = np.array([dataset[-PAST_HISTORY:,:]]) 
# predict next days
y_future_pred = multi_step_model.predict(x_for_future)

### Predict with TFlite & Compare 

#### Past days

In [17]:
# CONVERTED LITE MODEL
# load 
interpreter = tf.lite.Interpreter(model_content=tflite_model)

# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    # predict with tensorflow model
    expected = multi_step_model.predict(x_multi)
    # predict with TFlite model
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Reload Tlite model

In [18]:
interpreter = tf.lite.Interpreter(model_path=PATH_MDL_MULTI_TFLITE_FILE_KR)

In [19]:
PATH_MDL_MULTI_TFLITE_FILE_KR

'/Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/converted_model.tflite'

### Predict reloaded model

#### Past days

In [20]:
# Run the model with TensorFlow Lite
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check if same results 
for x_multi in list_x:
    
    expected = multi_step_model.predict(x_multi)
    
    interpreter.set_tensor(input_details[0]["index"], 
                           x_multi.astype(np.float32))
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=3)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    # Please note: TfLite fused Lstm kernel is stateful, so we need to reset
    # the states.
    # Clean up internal states.
    interpreter.reset_all_variables()

Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.
Done. The result of TensorFlow matches the result of TensorFlow Lite.


In [21]:
x_multi.shape

(1, 14, 8)

In [22]:
len(list_x)

9

## Update API lambda AWS

### API lambda simulate

#### Past days

In [23]:
dataset.shape

(434, 8)

In [24]:
json_list_list_x = prepare_to_lambda(dataset)
# simulate input to lambda (double dumps ? why ? i don't know yet)
json_list_list_x = json.dumps(json_list_list_x)
# simulate lambda

event = {"body": json_list_list_x}

[357 - 371]
[364 - 378]
[371 - 385]
[378 - 392]
[385 - 399]
[392 - 406]
[399 - 413]
[406 - 420]
[413 - 427]


In [25]:
# lambda code (file ./serverless/tensorflow-lite-on-aws-lambda/handler.py)
from serverless.tensorflow_lite_on_aws_lambda_kr.handler import predict
context = None
response = predict(event, context)


INPUT : nb. arr. : 9 / arrays shape: (1, 14, 8)
OUTPUT : nb. arrays : 9 / arrays shape in list: (1, 7)


In [26]:
# Retrieve from lambda in App code
# input : response
y_multi_pred_out = retrieve_from_lambda(response)      
y_multi_pred_out.shape

(1, 63)

In [27]:
y_multi_pred

array([[1.1467392 , 1.2568437 , 1.2005383 , 1.2311383 , 0.9787166 ,
        1.070845  , 1.0709375 , 1.1021686 , 1.173879  , 1.1427528 ,
        1.1675222 , 0.9826493 , 1.0500412 , 1.0485113 , 1.4185939 ,
        1.6137369 , 1.5244001 , 1.534508  , 1.1591265 , 1.3291066 ,
        1.3518373 , 1.3035349 , 1.4772024 , 1.3940609 , 1.4025687 ,
        1.0635716 , 1.2077435 , 1.2250835 , 1.1553187 , 1.3076409 ,
        1.2246069 , 1.2441945 , 0.9326531 , 1.0468261 , 1.0495495 ,
        1.2664165 , 1.4328315 , 1.3561014 , 1.3720568 , 1.0232157 ,
        1.1799825 , 1.1972817 , 1.0898176 , 1.1443336 , 1.137456  ,
        1.1562719 , 0.96160567, 1.0680109 , 1.076304  , 1.087279  ,
        1.1975048 , 1.1449891 , 1.162689  , 0.90126395, 1.0108123 ,
        1.0126414 , 1.0072403 , 1.0547724 , 1.0451384 , 1.0685495 ,
        0.88482314, 0.9759312 , 0.9773065 ]], dtype=float32)

In [28]:
y_multi_pred_out

array([[1.14673924, 1.25684345, 1.20053792, 1.23113811, 0.97871655,
        1.07084477, 1.0709374 , 1.10216868, 1.17387891, 1.14275265,
        1.16752219, 0.98264927, 1.05004108, 1.04851127, 1.418594  ,
        1.61373651, 1.52440012, 1.53450787, 1.15912652, 1.32910645,
        1.35183716, 1.30353475, 1.47720218, 1.39406061, 1.40256846,
        1.06357145, 1.20774364, 1.22508335, 1.15531886, 1.30764091,
        1.22460663, 1.24419439, 0.93265319, 1.04682624, 1.04954946,
        1.26641667, 1.43283141, 1.35610151, 1.37205672, 1.02321577,
        1.17998278, 1.19728196, 1.08981776, 1.1443336 , 1.13745606,
        1.15627193, 0.96160573, 1.06801093, 1.07630396, 1.08727908,
        1.19750476, 1.14498901, 1.16268885, 0.90126407, 1.0108124 ,
        1.01264167, 1.0072403 , 1.0547725 , 1.04513836, 1.06854975,
        0.88482332, 0.97593147, 0.97730654]])

In [29]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### Future days

In [30]:
# Prepare data to lambda (future)
json_list_list_x = prepare_to_lambda_future(dataset)

# simulate lambda
json_list_list_x = json.dumps(json_list_list_x) # dumps again : I dont know why
event = {"body": json_list_list_x}
context = None
response = predict(event, context)
y_future_pred_out = retrieve_from_lambda(response)      
y_future_pred_out.shape

INPUT : nb. arr. : 1 / arrays shape: (1, 14, 8)
OUTPUT : nb. arrays : 1 / arrays shape in list: (1, 7)


(1, 7)

In [31]:
y_future_pred_out

array([[0.87332934, 0.98735368, 0.9342795 , 0.95373517, 0.68816513,
        0.80265301, 0.80564356]])

In [32]:
y_future_pred

array([[0.8733294 , 0.9873537 , 0.93427956, 0.95373505, 0.6881651 ,
        0.80265313, 0.8056435 ]], dtype=float32)

In [33]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


### Update AWS Lambda with new model


This part does:
- Go to : ./serverless//tensorflow-lite-on-aws-lambda-kr
    
- Execute : sls deploy -v

In [34]:
str_exe = '#!/bin/bash\n' + \
    'export PATH="/usr/local/bin:$PATH"\n' + \
    f'cd {PATH_MDL_MULTI_TFLITE_KR}\n' + \
    'serverless deploy -v'
str_exe

'#!/bin/bash\nexport PATH="/usr/local/bin:$PATH"\ncd /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr\nserverless deploy -v'

In [35]:
open('deploy_serverless_kr.sh', "w").write(str_exe)
os.chmod('deploy_serverless_kr.sh', stat.S_IRWXU)

In [36]:
!cat ./deploy_serverless_kr.sh

#!/bin/bash
export PATH="/usr/local/bin:$PATH"
cd /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr
serverless deploy -v

In [37]:
!./deploy_serverless_kr.sh

Serverless: Deprecation warning: Resolution of lambda version hashes was improved with better algorithm, which will be used in next major release.
            Switch to it now by setting "provider.lambdaHashingVersion" to "20201221"
            More Info: https://www.serverless.com/framework/docs/deprecations/#LAMBDA_HASHING_VERSION_V2
Serverless: Generated requirements from /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/requirements.txt in /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/serverless/tensorflow_lite_on_aws_lambda_kr/.serverless/requirements.txt...
Serverless: Installing requirements from /Users/gregory/Library/Caches/serverless-python-requirements/eddadf68f9f2c21cf494701082a5393692d7b3c4711137913b3ed1b03bec68cf_slspyc/requirements.txt ...
Serverless: Docker Image: lambci/lambda:build-python3.8
Serverless: Using download cache 

### API AWS real Test

#### Past days

In [38]:
# prepare input
json_list_list_x = prepare_to_lambda(dataset)
# REQUEST
resp = requests.post(URL_PREDICT_KR, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

[357 - 371]
[364 - 378]
[371 - 385]
[378 - 392]
[385 - 399]
[392 - 406]
[399 - 413]
[406 - 420]
[413 - 427]
status code :  200
[[[1.1467392444610596, 1.2568434476852417, 1.200537919998169, 1.2311381101608276, 0.9787165522575378, 1.0708447694778442, 1.0709373950958252]], [[1.1021686792373657, 1.1738789081573486, 1.1427526473999023, 1.1675221920013428, 0.9826492667198181, 1.0500410795211792, 1.048511266708374]], [[1.4185940027236938, 1.6137365102767944, 1.5244001150131226, 1.5345078706741333, 1.1591265201568604, 1.3291064500808716, 1.351837158203125]], [[1.303534746170044, 1.4772021770477295, 1.3940606117248535, 1.4025684595108032, 1.0635714530944824, 1.2077436447143555, 1.225083351135254]], [[1.1553188562393188, 1.3076409101486206, 1.2246066331863403, 1.2441943883895874, 0.9326531887054443, 1.0468262434005737, 1.0495494604110718]], [[1.2664166688919067, 1.4328314065933228, 1.3561015129089355, 1.3720567226409912, 1.0232157707214355, 1.1799827814102173, 1.1972819566726685]], [[1.089817762

In [39]:
len(json_list_list_x)

20866

In [40]:
resp.json()

[[[1.1467392444610596,
   1.2568434476852417,
   1.200537919998169,
   1.2311381101608276,
   0.9787165522575378,
   1.0708447694778442,
   1.0709373950958252]],
 [[1.1021686792373657,
   1.1738789081573486,
   1.1427526473999023,
   1.1675221920013428,
   0.9826492667198181,
   1.0500410795211792,
   1.048511266708374]],
 [[1.4185940027236938,
   1.6137365102767944,
   1.5244001150131226,
   1.5345078706741333,
   1.1591265201568604,
   1.3291064500808716,
   1.351837158203125]],
 [[1.303534746170044,
   1.4772021770477295,
   1.3940606117248535,
   1.4025684595108032,
   1.0635714530944824,
   1.2077436447143555,
   1.225083351135254]],
 [[1.1553188562393188,
   1.3076409101486206,
   1.2246066331863403,
   1.2441943883895874,
   0.9326531887054443,
   1.0468262434005737,
   1.0495494604110718]],
 [[1.2664166688919067,
   1.4328314065933228,
   1.3561015129089355,
   1.3720567226409912,
   1.0232157707214355,
   1.1799827814102173,
   1.1972819566726685]],
 [[1.089817762374878,
   1.

In [41]:
y_multi_pred_out = retrieve_from_lambda(resp)      
y_multi_pred_out.shape

(1, 63)

In [42]:
y_multi_pred_out

array([[1.14673924, 1.25684345, 1.20053792, 1.23113811, 0.97871655,
        1.07084477, 1.0709374 , 1.10216868, 1.17387891, 1.14275265,
        1.16752219, 0.98264927, 1.05004108, 1.04851127, 1.418594  ,
        1.61373651, 1.52440012, 1.53450787, 1.15912652, 1.32910645,
        1.35183716, 1.30353475, 1.47720218, 1.39406061, 1.40256846,
        1.06357145, 1.20774364, 1.22508335, 1.15531886, 1.30764091,
        1.22460663, 1.24419439, 0.93265319, 1.04682624, 1.04954946,
        1.26641667, 1.43283141, 1.35610151, 1.37205672, 1.02321577,
        1.17998278, 1.19728196, 1.08981776, 1.1443336 , 1.13745606,
        1.15627193, 0.96160573, 1.06801093, 1.07630396, 1.08727908,
        1.19750476, 1.14498901, 1.16268885, 0.90126407, 1.0108124 ,
        1.01264167, 1.0072403 , 1.0547725 , 1.04513836, 1.06854975,
        0.88482332, 0.97593147, 0.97730654]])

In [43]:
y_multi_pred

array([[1.1467392 , 1.2568437 , 1.2005383 , 1.2311383 , 0.9787166 ,
        1.070845  , 1.0709375 , 1.1021686 , 1.173879  , 1.1427528 ,
        1.1675222 , 0.9826493 , 1.0500412 , 1.0485113 , 1.4185939 ,
        1.6137369 , 1.5244001 , 1.534508  , 1.1591265 , 1.3291066 ,
        1.3518373 , 1.3035349 , 1.4772024 , 1.3940609 , 1.4025687 ,
        1.0635716 , 1.2077435 , 1.2250835 , 1.1553187 , 1.3076409 ,
        1.2246069 , 1.2441945 , 0.9326531 , 1.0468261 , 1.0495495 ,
        1.2664165 , 1.4328315 , 1.3561014 , 1.3720568 , 1.0232157 ,
        1.1799825 , 1.1972817 , 1.0898176 , 1.1443336 , 1.137456  ,
        1.1562719 , 0.96160567, 1.0680109 , 1.076304  , 1.087279  ,
        1.1975048 , 1.1449891 , 1.162689  , 0.90126395, 1.0108123 ,
        1.0126414 , 1.0072403 , 1.0547724 , 1.0451384 , 1.0685495 ,
        0.88482314, 0.9759312 , 0.9773065 ]], dtype=float32)

In [44]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_multi_pred, y_multi_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.


#### future days

In [45]:
# prepare input
json_list_list_x = prepare_to_lambda_future(dataset)
resp = requests.post(URL_PREDICT_KR, json=json_list_list_x)
print("status code : ", resp.status_code) 
print(resp.json())

status code :  200
[[[0.8733293414115906, 0.9873536825180054, 0.9342795014381409, 0.9537351727485657, 0.6881651282310486, 0.8026530146598816, 0.8056435585021973]]]


In [46]:
y_future_pred_out = retrieve_from_lambda(resp)      
y_future_pred_out

array([[0.87332934, 0.98735368, 0.9342795 , 0.95373517, 0.68816513,
        0.80265301, 0.80564356]])

In [47]:
y_future_pred

array([[0.8733294 , 0.9873537 , 0.93427956, 0.95373505, 0.6881651 ,
        0.80265313, 0.8056435 ]], dtype=float32)

In [48]:
# Assert if the result of TFLite model is consistent with the TF model.
np.testing.assert_almost_equal(y_future_pred, y_future_pred_out, decimal=3)
print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

Done. The result of TensorFlow matches the result of TensorFlow Lite.
